<h1 style="text-align:center;font-size:50px;" > Self driving car </h1>

# 1.0 Function to load data in batches

In [10]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 331kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.2
    Uninstalling scipy-1.3.2:
      Successfully uninstalled scipy-1.3.2


In [0]:
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)] # train-test split- 70:30
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out

45406


In [1]:
!python driving_data.py

# 2.0 Defining model architecture

In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.identity(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #using identity functiona as activation

In [2]:
!python model.py

C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# 3.0 Training the model

In [ ]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
from scipy.misc import imread

LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss) #1e-4 suggested by the team
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.5}) #dropout rate= 50%
        if i % 10 == 0:
            xs, ys = driving_data.LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


In [14]:
!python train.py

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-12-04 06:53:18.651429: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-12-04 06:53:18.653532: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1d89640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-04 06:53:18.653571: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-04 06:53:18.675045: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-04 06:53:18.864171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-12

# 4.0 Testing & results

In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.misc
import model
import cv2
from subprocess import call
import math
from scipy.misc import imread


sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()

In [3]:
!python run_dataset.py

Starting frameofvideo:31785
Steering angle: 0.5939997808880295 (pred)	-28.34 (actual)
Steering angle: 0.5276790709340289 (pred)	-28.840000000000003 (actual)
Steering angle: 0.7149999971293904 (pred)	-29.75 (actual)
Steering angle: 0.9039798055194188 (pred)	-31.06 (actual)
Steering angle: 1.0341948007247732 (pred)	-32.27 (actual)
Steering angle: 1.0598255124720692 (pred)	-33.48 (actual)
Steering angle: 1.0624794678514422 (pred)	-34.39 (actual)
Steering angle: 1.1170842689447555 (pred)	-35.599999999999994 (actual)
Steering angle: 1.2334288574931065 (pred)	-36.5 (actual)
Steering angle: 1.1999387324452413 (pred)	-37.61 (actual)
Steering angle: 1.3446525117112402 (pred)	-38.62 (actual)
Steering angle: 1.4113324469269013 (pred)	-39.63 (actual)
Steering angle: 1.5388979690645255 (pred)	-39.93000000000001 (actual)
Steering angle: 1.4749187333353697 (pred)	-40.03000000000001 (actual)
Steering angle: 1.6364597508387528 (pred)	-40.03000000000001 (actual)
Steering angle: 1.5633457774249397 (pred)

Steering angle: -0.7024375716984445 (pred)	-39.53 (actual)
Steering angle: -0.7309971539392065 (pred)	-39.53 (actual)
Steering angle: -0.5843077424557814 (pred)	-39.53 (actual)
Steering angle: -0.5520846174989538 (pred)	-38.81999999999999 (actual)
Steering angle: -0.3731353687919122 (pred)	-37.92 (actual)
Steering angle: -0.36309285627787785 (pred)	-37.82 (actual)
Steering angle: -0.2206667634970101 (pred)	-37.71 (actual)
Steering angle: -0.1912431623262414 (pred)	-37.31 (actual)
Steering angle: -0.11096922967155091 (pred)	-36.81 (actual)
Steering angle: -0.11601204371313595 (pred)	-36.90999999999999 (actual)
Steering angle: -0.08554342359935532 (pred)	-37.21 (actual)
Steering angle: 0.07971897978462694 (pred)	-37.21 (actual)
Steering angle: 0.2220071881216251 (pred)	-37.21 (actual)
Steering angle: 0.3597810683040679 (pred)	-37.31 (actual)
Steering angle: 0.48475560086873 (pred)	-37.41 (actual)
Steering angle: 0.6629535287670701 (pred)	-37.41 (actual)
Steering angle: 0.7599080640354011

Steering angle: 0.8947406940065072 (pred)	-72.71 (actual)
Steering angle: 0.8922113895794869 (pred)	-73.31 (actual)
Steering angle: 0.41692669976862584 (pred)	-74.21999999999998 (actual)
Steering angle: 0.0751973945540785 (pred)	-74.92000000000002 (actual)
Steering angle: 0.11306012133122106 (pred)	-75.53000000000002 (actual)
Steering angle: -0.04929817099503641 (pred)	0.0 (actual)
Steering angle: -0.02794187700769904 (pred)	-75.83 (actual)
Steering angle: -0.021339645401235462 (pred)	-75.83 (actual)
Steering angle: -0.2542644640243028 (pred)	-75.83 (actual)
Steering angle: -0.15276699918426706 (pred)	-75.83 (actual)
Steering angle: 0.11695546359224396 (pred)	-75.83 (actual)
Steering angle: 0.042642151648858984 (pred)	-75.23 (actual)
Steering angle: 0.2284894645307903 (pred)	-74.52 (actual)
Steering angle: 0.39980981882175076 (pred)	-73.81999999999998 (actual)
Steering angle: 0.5151559190908105 (pred)	-73.71 (actual)
Steering angle: 0.4020693307765556 (pred)	-73.61 (actual)
Steering an

Steering angle: 1.3764086224835683 (pred)	-19.36 (actual)
Steering angle: 1.438137737768028 (pred)	-19.56 (actual)
Steering angle: 1.3945713761471017 (pred)	-19.76 (actual)
Steering angle: 1.3563410998152476 (pred)	-19.97 (actual)
Steering angle: 1.3294322889195131 (pred)	-20.17 (actual)
Steering angle: 1.331071534320316 (pred)	-20.370000000000005 (actual)
Steering angle: 1.338495523061297 (pred)	-20.469999999999995 (actual)
Steering angle: 1.3244560692223357 (pred)	-20.67 (actual)
Steering angle: 1.3367115630274542 (pred)	-20.97 (actual)
Steering angle: 1.3453205895894322 (pred)	-21.18 (actual)
Steering angle: 1.399723473215407 (pred)	-21.18 (actual)
Steering angle: 1.5035594240725256 (pred)	-21.18 (actual)
Steering angle: 1.5118988716056996 (pred)	-21.18 (actual)
Steering angle: 1.4841949840018933 (pred)	-21.18 (actual)
Steering angle: 1.4936447641620174 (pred)	-21.18 (actual)
Steering angle: 1.4566253523039598 (pred)	-21.18 (actual)
Steering angle: 1.5675623520203648 (pred)	-21.18 (

Steering angle: -0.22805532063167136 (pred)	-6.659999999999999 (actual)
Steering angle: -0.26529991528892777 (pred)	-6.659999999999999 (actual)
Steering angle: -0.23146401191433086 (pred)	-6.659999999999999 (actual)
Steering angle: -0.13166854483809767 (pred)	-6.76 (actual)
Steering angle: -0.05919255378144613 (pred)	-6.659999999999999 (actual)
Steering angle: -0.24557262141863845 (pred)	-6.76 (actual)
Steering angle: -0.28507587425705344 (pred)	-6.76 (actual)
Steering angle: -0.22888348106853543 (pred)	-6.76 (actual)
Steering angle: -0.3291737099727714 (pred)	-6.76 (actual)
Steering angle: -0.33768071058402227 (pred)	-6.76 (actual)
Steering angle: -0.24947180566106947 (pred)	-6.659999999999999 (actual)
Steering angle: -0.3376448520908797 (pred)	-6.76 (actual)
Steering angle: -0.13847568211966163 (pred)	-6.659999999999999 (actual)
Steering angle: -0.42786652838487066 (pred)	-6.76 (actual)
Steering angle: -0.5017631987888849 (pred)	-6.659999999999999 (actual)
Steering angle: -0.51856887

...

Steering angle: 1.1855710026392965 (pred)	-154.18 (actual)
Steering angle: 1.1835411557953333 (pred)	-154.89 (actual)
Steering angle: 1.2155316274335572 (pred)	-155.39 (actual)
Steering angle: 1.1973291732954734 (pred)	-155.39 (actual)
Steering angle: 1.2134817169089074 (pred)	-155.39 (actual)
Steering angle: 1.283124887007296 (pred)	-155.39 (actual)
Steering angle: 1.378979334932432 (pred)	-155.29 (actual)
Steering angle: 1.5423604480868986 (pred)	0.0 (actual)
Steering angle: 1.4791641227913557 (pred)	-152.47 (actual)
Steering angle: 1.291418444206984 (pred)	-150.25000000000003 (actual)
Steering angle: 1.2186368021849745 (pred)	-148.24 (actual)
Steering angle: 1.186391052226521 (pred)	-146.52 (actual)
Steering angle: 1.1361084802173567 (pred)	-144.4 (actual)
Steering angle: 1.0106989499799293 (pred)	-144.2 (actual)
Steering angle: 1.2891077058334042 (pred)	-142.99 (actual)
Steering angle: 1.3198456915325771 (pred)	-141.48 (actual)
Steering angle: 1.325814849980345 (pred)	-140.26999999

Steering angle: 1.0118331882689746 (pred)	-14.420000000000002 (actual)
Steering angle: 0.9719508599316239 (pred)	-14.420000000000002 (actual)
Steering angle: 0.9730227727444928 (pred)	-14.420000000000002 (actual)
Steering angle: 0.9819494031028763 (pred)	-14.72 (actual)
Steering angle: 1.00028974168483 (pred)	-15.030000000000001 (actual)
Steering angle: 1.0110408863252531 (pred)	-15.43 (actual)
Steering angle: 1.0523340756130346 (pred)	-15.83 (actual)
Steering angle: 1.0526337501628689 (pred)	-15.929999999999998 (actual)
Steering angle: 1.085428476575862 (pred)	-15.929999999999998 (actual)
Steering angle: 1.0761940608180047 (pred)	-15.929999999999998 (actual)
Steering angle: 1.0377418033381254 (pred)	-15.730000000000002 (actual)
Steering angle: 0.9968665362501842 (pred)	-15.63 (actual)
Steering angle: 0.9650032768852516 (pred)	-15.63 (actual)
Steering angle: 0.9584710547177806 (pred)	-15.83 (actual)
Steering angle: 0.9547874483210281 (pred)	-15.929999999999998 (actual)
Steering angle: 

Steering angle: 1.6939249070904205 (pred)	9.28 (actual)
Steering angle: 1.6923930237256335 (pred)	9.28 (actual)
Steering angle: 1.7498093014360532 (pred)	9.28 (actual)
Steering angle: 1.7792926667620594 (pred)	9.28 (actual)
Steering angle: 1.7882286886305518 (pred)	9.28 (actual)
Steering angle: 1.7663650396539294 (pred)	9.28 (actual)
Steering angle: 1.7783844650458593 (pred)	9.28 (actual)
Steering angle: 1.7647834239742484 (pred)	9.28 (actual)
Steering angle: 1.7536071000579918 (pred)	9.28 (actual)
Steering angle: 1.7686277532599077 (pred)	9.28 (actual)
Steering angle: 1.7447351112129683 (pred)	9.28 (actual)
Steering angle: 1.7089024447232006 (pred)	9.28 (actual)
Steering angle: 1.7008146469722591 (pred)	9.28 (actual)
Steering angle: 1.6873619490716845 (pred)	9.28 (actual)
Steering angle: 1.679737536967246 (pred)	9.28 (actual)
Steering angle: 1.6877388901365045 (pred)	9.28 (actual)
Steering angle: 1.683381869776271 (pred)	9.28 (actual)
Steering angle: 1.6931334589203453 (pred)	9.28 (ac

Steering angle: 1.4185082009802343 (pred)	14.12 (actual)
Steering angle: 1.419153440413389 (pred)	14.12 (actual)
Steering angle: 1.3698285889919073 (pred)	14.12 (actual)
Steering angle: 1.3617698195449386 (pred)	14.32 (actual)
Steering angle: 1.271675355524239 (pred)	14.72 (actual)
Steering angle: 1.2745436080888213 (pred)	15.83 (actual)
Steering angle: 1.2564538520720425 (pred)	16.24 (actual)
Steering angle: 1.2366702091411006 (pred)	16.64 (actual)
Steering angle: 1.2335278952360715 (pred)	16.64 (actual)
Steering angle: 1.2554463991694655 (pred)	16.64 (actual)
Steering angle: 1.268393876514871 (pred)	16.64 (actual)
Steering angle: 1.285340429619319 (pred)	16.64 (actual)
Steering angle: 1.2871632363540662 (pred)	16.64 (actual)
Steering angle: 1.2951276638131242 (pred)	16.64 (actual)
Steering angle: 1.2897335219161066 (pred)	16.64 (actual)
Steering angle: 1.2452420965494666 (pred)	16.64 (actual)
Steering angle: 1.2560115973232842 (pred)	16.64 (actual)
Steering angle: 1.2528765404942483 

C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-12-04 14:17:02.817499: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
Traceback (most recent call last):
  File "run_dataset.py", line 45, in <module>
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
IndexError: list index out of range


# 5.0 Conclusion

The results were not satisfactory due to the following reasons:
- Identity function was used in the output layer instead of atan function
- The dropout rate of 0.5 was more